In [4]:
'''
This script is used to extract text from the voter ID and stored in testfile.txt
'''
# libs
import cv2
import numpy as np
import pytesseract
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def get_string(img_path):
    # Read image with opencv
    img = cv2.imread(img_path)

    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion to remove some noise
    kernel = np.ones((7, 6), np.uint8)
    #print(kernel)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)

    # Write image after removed noise
    cv2.imwrite("removed_noise.png", img)

    # Write the image after apply opencv to do some
    cv2.imwrite('sample_ritesh_process.png', img)
    
    # Recognize text with tesseract for python
    result = pytesseract.image_to_string(Image.open('sample_ritesh_process.png'))

    # Writing Result into text file 
    file = open("testfile.txt","w") 
    file.write(result)
    file.close()

    return result

get_string('sample_ritesh.png')
print ("------ Done -------")

------ Done -------


In [5]:
import json
import re

# Open and reading the textfile containing result
filename = open('testfile.txt', 'r')
text = filename.read()

text1 = []

# Splitting the lines to sort the text paragraph wise
lines = text.split('\n')
for lin in lines:
    s = lin.strip()
    s = s.rstrip()
    s = s.lstrip()
    text1.append(s)
print(text1) 

['em nahh hee —', '£', '', 'es', 'ELECTION COMMISSION OF INDIA', '', 'IDENTITY CARD', 'AVE4499919', '', ': Teter aay fe wage', '', "Elector's Name : Ritesh Dasharath Sinh", 'Rajput', '', "Father's Name: Dasharath Sinh Rajput", '', 'eT / Sex - YoW/ Male', 'WAAR / Date of Birth : 13/12/1996']


In [6]:
'''
This script is used to structur the text obtained from voter id, it is saved in result.json file 
'''
# libs
import json
import re

# Open and reading the textfile containing result
filename = open('testfile.txt', 'r')
text = filename.read()

text1 = []

# Splitting the lines to sort the text paragraph wise
lines = text.split('\n')
for lin in lines:
    s = lin.strip()
    s = s.rstrip()
    s = s.lstrip()
    text1.append(s)

# Using regex to find the neceesary information
def findword(textlist, wordstring):
    lineno = -1
    for wordline in textlist:
        xx = wordline.split()
        if ([w for w in xx if re.search(wordstring, w)]):
            lineno = textlist.index(wordline)
            textlist = textlist[lineno+1:]
            return textlist
    return textlist

# Finding the electors number 
voter_no = findword(text1, '(ELECTION COMMISSION OF INDIA IDENTITY CARD|CARD|IDENTITY CARD)$')
voter_no = voter_no[0]
epic_no = voter_no.replace(" ", "")
print('\n')
print('Epic_No:',epic_no)

# Some voter id's last name is printed on next line hence, it will extract from next line
find_word = "(Elector's Name|NAME|Name)$"
name_end = findword(text1, find_word)
endname = name_end[0]

lines = text
for x in lines.split('\n'):
    _ = x.split()
    if ([w for w in _ if re.search("(Elector's Name|ELECTOR'S NAME|NAME|Name|name)$", w)]):
        person_name = x
        person_name = person_name.split(':')[1].strip()
        
        # If voter id's endname is on next line it will join it
        if endname:
            print("Elector's Name:",person_name + ' ' + endname)
            full_name = person_name + ' ' + endname
        else:
            print(person_name)
            full_name = person_name

    # Finding the father/husband/mother name        
    if ([w for w in _ if re.search("(Father's|Mother's|Husband's)$", w)]):
        elder_name = x
        elder_name = elder_name.split(':')[1].strip()
        print("Father's Name:",elder_name)
        
    # Finding the gender of the electoral candidate    
    if ([w for w in _ if re.search('(sex|SEX|Sex)$', w)]):
        gender = x
        gender = gender.split('/')
        sex = ''.join(gender[2]).strip()
        print('Sex:',sex)
    
    # Finding the Date of Birth 
    if ([w for w in _ if re.search('(Year|Birth|Date of Birth|DATE OF BIRTH|DOB)$', w)]):
        year = x
        year = year.split(':')
        dob = ''.join(year[1:]).strip()
        print('Date of Birth:',dob)

# Converting the extracted informaton into json
di = {'Epic_No':epic_no,
      'Elector Name':full_name,
      'Father Name':elder_name,
      'Sex':sex,
      'Date of Birth':dob}

# Saving the json file
print('\n',di)
with open('result.json', 'w') as fp:
    json.dump(di, fp, sort_keys=True, indent=4)



Epic_No: AVE4499919
Elector's Name: Ritesh Dasharath Sinh Rajput
Father's Name: Dasharath Sinh Rajput
Sex: Male
Date of Birth: 13/12/1996

 {'Epic_No': 'AVE4499919', 'Elector Name': 'Ritesh Dasharath Sinh Rajput', 'Father Name': 'Dasharath Sinh Rajput', 'Sex': 'Male', 'Date of Birth': '13/12/1996'}


In [7]:
'''
This script is used to get info from https://electoralsearch.in/##resultArea  
Website : https://electoralsearch.in/##resultArea
'''
# Lib
import json
import time
from pytesseract import image_to_string 
from PIL import Image
import pytesseract
from selenium import webdriver
from bs4 import BeautifulSoup as BSoup

def get_captcha_text(location, size):
    
    ''' Edit this path to your stored tesseract.exe path '''
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    im = Image.open('screenshot.png')
    left = location['x'] + 320
    top = location['y'] + 250
    right = location['x'] + size['width'] + 220
    bottom = location['y'] + size['height'] + 100
    
    # Define crops points
    im = im.crop((left, right, top, bottom))
    im.save('screenshot.png')
    captcha_text = image_to_string(Image.open('screenshot.png'))
    print('CAPTCHA:',captcha_text)
    return captcha_text

def login_to_website():
    # Extracting epic_no of the voter from the json file
    with open('result.json') as f:
        file = json.load(f)
    epic_no = file['Epic_No']
    
    url = 'https://electoralsearch.in/##resultArea'
    
    ''' Edit the path as per your chromedriver.exe path '''
    driver = webdriver.Chrome(executable_path = r"C:\Program Files\chromedriver.exe")
    print('----------Opening browser----------')
    time.sleep(5)
    driver.get(url)
    driver.set_window_size(1120, 550)
    driver.find_element_by_xpath('//*[@id="continue"]').click()

    # Click on the second tab i.e. epic_no of the webpage
    driver.find_element_by_xpath('//*[@id="mainContent"]/div[2]/div/div/ul/li[2]').click() 
    
    # Screenshot the image of screen and determines location of the captcha
    element = driver.find_element_by_xpath('//*[@id="imgCaptchaDiv"]/img')
    location = element.location
    size = element.size        

    # Name of the voter
    user_id = driver.find_element_by_xpath('//*[@id="name"]')
    user_id.clear()
    user_id.send_keys(epic_no)

    # State of the Voter
    password = driver.find_element_by_xpath('//*[@id="epicStateList"]')
    password.send_keys('Maharashtra')

    voter_details = driver.find_elements_by_xpath('//*[@id="resultsTable"]/tbody/tr/td[1]/form/input[22]')

    # Until we obtain the result it will solve the captcha in loop and break when it founds the result 
    while not (voter_details):
        driver.save_screenshot('screenshot.png')
        captcha = driver.find_element_by_xpath('//*[@id="txtEpicCaptcha"]')
        captcha_text = get_captcha_text(location, size)
        time.sleep(5)
        captcha.send_keys(captcha_text)
        time.sleep(5)
        driver.find_element_by_xpath('//*[@id="btnEpicSubmit"]').click()
        time.sleep(7)
        voter_details = driver.find_elements_by_xpath('//*[@id="resultsTable"]/tbody/tr/td[1]/form/input[22]')        
        if voter_details:
            break
        print('----------Captcha not solved, Please wait----------')
    print('----------DONE!!!----------')
    time.sleep(10)
    driver.find_element_by_xpath('//*[@id="resultsTable"]/tbody/tr/td[1]/form/input[22]').click
    
    # Parsing the page through BeautifulSoup
    bs_obj = BSoup(driver.page_source, 'html.parser')
    
    # Finding the detials of the Voter
    for i in bs_obj.select('tbody'):
        text = i.get_text()
    cleanedtext = text.strip()
    if cleanedtext: 
        print(cleanedtext)    
    
    # Saving it in a text file
    file = open("finaltext.txt","w", encoding='utf-8') 
    file.write(cleanedtext)
    file.close()
      
login_to_website()

----------Opening browser----------
CAPTCHA: TV5GIr
----------DONE!!!----------
AVE4499919
Ritesh Dasharath Sinh Rajput रीतेश दशरथ सिंह राजपूत
22
Dasharath Sinh Rajput दशरथ सिंह राजपूत
Maharashtra
Mumbai Suburban


Kapol Vidyalaya Mahavir Nagar, 1st  Floor Room No 4 Kandivali (W)x

Charkop
Mumbai North
